# 5. Prédire les maladies cardiaques

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Nous travaillons sur un jeu de données fourni par le projet __[drivendata](https://www.drivendata.org)__ : __[Warm Up: Machine Learning with a Heart](https://www.drivendata.org/competitions/54/machine-learning-with-a-heart/)__

In [ ]:
df = pd.read_csv('/data/heart_values.csv')
df_label = pd.read_csv('/data/heart_labels.csv')
df = pd.merge(df, df_label, on='patient_id')

## 5.1 Analyse exploratoire des données

Explorez ce dataset.

In [ ]:
print(df.shape)
df.head()

In [ ]:
df.describe()

In [ ]:
df.info()

Analyse des corrélations :

In [ ]:
plt.figure(figsize=(6, 5))
sns.heatmap(df.corr())
plt.show()

Modalités des variables catégorielles :

In [ ]:
for col in df.select_dtypes(include=[object]).columns:
    counts = df[col].value_counts()
    if len(counts) < 20:
        print(df[col].value_counts(), "\n")
    else:
        print("{} has {} unique values\n".format(col, len(counts)))

In [ ]:
df.plot.kde(
    subplots=True,
    layout=(5, 3),
    legend=False,
    title=df.dtypes[df.dtypes != 'object'].index.tolist(),
    figsize=(20, 15)
);

In [ ]:
df.columns
df.dtypes[df.dtypes != 'object']

Distribution des valeurs pour chaque variable numérique :

In [ ]:
df.hist(figsize=(20, 15));

## 5.2 Classement des variables en fonction de leurs types

Ce dataset contient des variables de plusieurs types (numériques, catégorielles et ordinales). Nous devons appliquer des pre-traitements différents pour chaque type de variable.

Identifiez les types pour chaque variable et construisez trois listes Python contenant le nom des variables pour trois types : `var_names_num` (type numérique) `var_names_cat` (type catégoriel), `var_names_ord` (type ordinal). La variable `age` sera traitée différement (i.e. : ne l'incluez dans aucune de ces listes) :

In [ ]:
list(df)
df.columns

In [ ]:
var_names_num = ['max_heart_rate_achieved', 'oldpeak_eq_st_depression', 
                 'resting_blood_pressure', 'serum_cholesterol_mg_per_dl']
var_names_cat = ['thal', 'exercise_induced_angina', 'sex',
                 'fasting_blood_sugar_gt_120_mg_per_dl',
                 'resting_ekg_results', 'slope_of_peak_exercise_st_segment']
var_names_ord = ['num_major_vessels', 'chest_pain_type']

Les variables numériques seront standardisées, mais la classe `StandardScaler` ne peut travailler que sur des nombres à virgule flottante. Transformez chacune des variables numérique de type `int64` en `float64` à l'aide de la méthode [pandas.DataFrame.astype](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html) :

In [ ]:
#df['max_heart_rate_achieved'] = df.max_heart_rate_achieved.astype(np.float64)
for col in var_names_num:
    df[col] = df[col].astype(np.float64)

In [ ]:
df.dtypes

In [ ]:
df = df.astype(dict([(var,np.float64) for var in var_names_num]));

## 5.3 Construction du Jeu d'entraînement et du jeu de test

Construisez un jeu d'entrainement et un jeu de test contenant 20% de notre dataset, pensez bien à séparer votre cible `heart_disease_present`. Pour la suite de ce notebook, vos variables devraient être nommées `X_train`, `y_train`, `X_test` et `y_test` :

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=77)
X_train = train.drop("heart_disease_present", axis=1)
y_train = train["heart_disease_present"].copy()
X_test = test.drop("heart_disease_present", axis=1)
y_test = test["heart_disease_present"].copy()

## 5.4 Transformation des variables

Nous allons créer une chaîne de transformation pour préparer nos données à l'aide de la classe [`sklearn.pipeline.Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html).

Commencons par les features numériques : créez une instance de `Pipeline` en lui passant comme argument une liste composée d'un tuple de deux valeurs dont la première est un nom simple à retenir (par exemple : `std_scaler`) et la deuxième une instance de [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Nommez cette instance `num_pipeline` :

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
num_pipeline = Pipeline([('std_scaler', StandardScaler())])

Faites de même pour :
* les features ordinales avec [`sklearn.preprocessing.OrdinalEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html), nom de l'étape du Pipeline : `ord_encoder`, nom de l'instance `ord_pipeline`
* les features catégorielles avec [`sklearn.preprocessing.OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) (passez la valeur `'ignore'` au paramètre `handle_unknown`), nom de l'étape du Pipeline : `1h_encoder`, nom de l'instance `cat_pipeline`

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

ord_pipeline = Pipeline([('ord_encoder', OrdinalEncoder())])
cat_pipeline = Pipeline([('1h_encoder', OneHotEncoder(handle_unknown='ignore'))])

Créez un pipeline pour la variable `age`, que vous nommerez `bin_discretizer` et que vous stockerez dans une variable `age_pipeline`, avec [`sklearn.preprocessing.KBinsDiscretizer`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.KBinsDiscretizer.html). Vous pouvez passer en paramètres `n_bins=6, strategy='uniform', encode='ordinal'` :

In [ ]:
from sklearn.preprocessing import KBinsDiscretizer

age_pipeline = Pipeline(
    [
        ('bin_discretizer', KBinsDiscretizer(n_bins=6,
                                             strategy='uniform',
                                             encode='ordinal'
                                            )
        )
    ]
)

Pour finir nous allons composer nos Pipeline en indiquant sur quelles variables ils s'appliquent.

Créez une instance de [`sklearn.compose.ColumnTransformer`](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) en lui passant en paramètre une liste de tuples de trois valeurs. La première sera un nom simple à retenir (`num`, `ord`, `cat` et `age` par exemple), la deuxième votre variable contenant le pipeline correspondant et la dernière la liste des colonnes sur lesquels ce pipeline doit s'appliquer :

In [ ]:
from sklearn.compose import ColumnTransformer

preparation_pipeline = ColumnTransformer([
    ("num", num_pipeline, var_names_num),
    ("ord", ord_pipeline, var_names_ord),
    ("cat", cat_pipeline, var_names_cat),
    ("age", age_pipeline, ['age'])
])

Visualisez l'effet de cette étape de transformation en l'appliquant sur `X_train` par exemple avec la méthode `fit_transform` (vous obtenez un tableau numpy, vous pouvez donc sélectionner la première ligne pour simplifier l'affichage) :

In [ ]:
preparation_pipeline.fit_transform(X_train)[0]

## 5.5 Apprentissage d'une régression logistique

Créez un Pipeline comprenant deux étapes :
* vos pré-traitement, nommé `preparation`
* une instance de [`sklearn.linear_model.LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), nommé `model` avec les paramètres `solver='liblinear', penalty='l2'` :

In [ ]:
from sklearn.linear_model import LogisticRegression

log_reg_pipeline = Pipeline([
    ("preparation", preparation_pipeline),
    ("model", LogisticRegression(solver='liblinear', penalty='l2'))
])

In [ ]:
import sklearn
sklearn.__version__

Créez une grille de paramètre pour ce pipeline où vous ferez varier le paramètre `C` de la régression logistique (`np.logspace(-5, 5, 10)` par exemple), et le paramètre `n_bins` de la binarisation de l'age (`[3,6,12]` par exemple). Les paramètres dans un pipeline sont accessible en précisant le nom de chaque sous-partie du pipeline séparé par deux underscores. Par exemple, `preparation__age__bin_discretizer__n_bins` permet de préciser que nous souhaitons faire varier le paramètre `n_bins` de `bin_discretizer`, lui même étant dans `age`, lui même dans `preparation` (d'où l'importance de bien nommé ses différentes étapes de pipeline) :

In [ ]:
param_grid = [
    {
        'model__C': np.logspace(-5, 5, 10),
        'preparation__age__bin_discretizer__n_bins': [3,6,12]
    }
]

Appliquez un grid search sur votre espace de recherche en utilisant la métrique de performance `f1` (paramètre `scoring` de `GridSearchCV` :

In [ ]:
from sklearn.model_selection import GridSearchCV

grid_search = GridSearchCV(
    log_reg_pipeline, param_grid, cv=5,
    scoring="f1"
)

grid_search.fit(X_train, y_train);

Affichez les meilleurs paramètres et le score correspondant :

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

Calculez les scores avec une validation croisée pour le meilleur modèle sur le jeu d'entrainement :

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, scoring="f1", cv=10)
np.mean(scores), np.std(scores)

## 5.6 Apprentissage d'un forêt aléatoire

Faites de même avec l'estimateur [`sklearn.ensemble.RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) (exemples pour le grid search : `max_depth` : `np.linspace(10,50,5, dtype=int)`, `min_samples_leaf` : `[2,4,8]`). Quel est le meilleur modèle ?

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_pipeline = Pipeline([
    ("preparation", preparation_pipeline),
    ('model', RandomForestClassifier(n_jobs=-1, n_estimators=100, random_state=77))
])

param_grid = {
    'model__max_depth': np.linspace(10,50,5, dtype=int),
    'model__min_samples_leaf': [2,4,8],
    'preparation__age__bin_discretizer__n_bins': [3,6,12]
}

grid_search = GridSearchCV(
    rf_pipeline, param_grid, cv=5,
    scoring="f1"
)

grid_search.fit(X_train, y_train);

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_score_

In [ ]:
scores = cross_val_score(grid_search.best_estimator_, X_train, y_train, scoring="f1", cv=10)
np.mean(scores), np.std(scores)

Le meilleur modèle est obtenu avec une forêt aléatoire et les hyper-paramètres `max_depth = 10`, `min_samples_leaf = 4` et `age_bin_discretizer_n_bins = 3`.

## 5.7 Analyse des résultats pour le meilleur modèle

Entrainer votre meilleur modèle sur tout le dataset d'entrainement, puis prédisez les classes pour votre dataset de test. Ensuite appliquez les fonctions `precision_score, recall_score, f1_score` du module [`sklearn.metrics`](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics). Quel type d'erreurs fait votre classifieur ?

In [ ]:
best_model = grid_search.best_estimator_.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

y_pred = best_model.predict(X_test)

print("Précision : ", precision_score(y_test, y_pred))
print("Rappel : ", recall_score(y_test, y_pred))
print("F1 : ", f1_score(y_test, y_pred))

Affichez la matrice de confusion avec [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html). Vous pouvez obtenir un graphique de cette matrice avec [`sklearn.metrics.plot_confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.plot_confusion_matrix.html) ou [`sklearn.metrics.ConfusionMatrixDisplay.from_predictions`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html#sklearn.metrics.ConfusionMatrixDisplay.from_predictions) :

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
print(y_pred)
print(y_test.values)

In [ ]:
from sklearn.metrics import plot_confusion_matrix

plot_confusion_matrix(best_model, X_test, y_test)  
plt.show()

In [ ]:
from sklearn import set_config
set_config(display='diagram')
#rf_pipeline

In [ ]:
import sklearn
sklearn.__version__

## 5.8 Arbre de décision

Le meilleur modèle obtenu est difficilement compréhensible (par des experts métier par exemple). Nous pouvons essayer d'entrainer un modèle plus simple et interprétable : un arbre de décision.

Entrainez un arbre de décision sur les seules variables `max_heart_rate_achieved` et `oldpeak_eq_st_depression` avec une profondeur maximale de deux. Calculez la performance du modèle obtenu. Puis afficher cet arbre avec [`sklearn.tree.export_graphviz`](https://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html). Vous pouvez enregistrer la sortie dans un fichier `.dot` et afficher le résultats avec le transformer en image avec le service : http://webgraphviz.com/

Essayez d'améliorer le score obtenu avec un arbre de décision tout en conservant un modèle interprétable (features utilisées et profondeur maximale).

In [ ]:
from sklearn.tree import DecisionTreeClassifier

X = X_train[['max_heart_rate_achieved','oldpeak_eq_st_depression']]
y = y_train

tree = DecisionTreeClassifier(max_depth=2, random_state=77)
tree.fit(X,y)

In [ ]:
y_pred = tree.predict(X_test[['max_heart_rate_achieved','oldpeak_eq_st_depression']])
print("Précision : ", precision_score(y_test, y_pred))
print("Rappel : ", recall_score(y_test, y_pred))
print("F1 : ", f1_score(y_test, y_pred))

In [ ]:
from sklearn.tree import export_graphviz

export_graphviz(
    tree,
    out_file="heart_tree.dot",
    feature_names=['max_heart_rate_achieved','oldpeak_eq_st_depression'],
    class_names=['malade', 'non malade'],
    rounded=True,
    filled=True
)

Le fichier `.dot` généré par `export_graphviz` est transformé en image par le service : http://webgraphviz.com/

![tree_graph](heart_tree.png)

La fonction suivante permet d'afficher les frontières de décision de l'arbre de décision sur notre dataset :

In [ ]:
def plot_decision_boundary(tree, X, y, axes):
    from matplotlib.colors import ListedColormap
    def make_grid_coord(x1_min, x1_max, x2_min, x2_max):
        x1s = np.linspace(x1_min, x1_max, 100)
        x2s = np.linspace(x2_min, x2_max, 100)
        x1, x2 = np.meshgrid(x1s, x2s)
        return (x1, x2, np.c_[x1.ravel(), x2.ravel()])

    (x1, x2, X_new) = make_grid_coord(axes[0], axes[1], axes[2], axes[3])
    y_pred = tree.predict(X_new).reshape(x1.shape)
    custom_cmap = ListedColormap(['#fafab0','#9898ff','#a0faa0'])
    plt.contourf(x1, x2, y_pred, alpha=0.3, cmap=custom_cmap)
    plt.plot(X.iloc[:,0][y==0], X.iloc[:,1][y==0], "yo", label="Non")
    plt.plot(X.iloc[:,0][y==1], X.iloc[:,1][y==1], "bs", label="Oui")
    plt.xlabel(r"$x_1$", fontsize=18)
    plt.ylabel(r"$x_2$", fontsize=18, rotation=0)

In [ ]:
plt.figure(figsize=(8, 4))
plot_decision_boundary(tree, X, y, [90, 210, -1, 7])